<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Supervised Learning Model Comparison

---

### Let us begin...

Recall the `data science process`.
   1. Define the problem.
   2. Gather the data.
   3. Explore the data.
   4. Model the data.
   5. Evaluate the model.
   6. Answer the problem.

In this lab, we're going to focus mostly on creating (and then comparing) many regression and classification models. Thus, we'll define the problem and gather the data for you.
Most of the questions requiring a written response can be written in 2-3 sentences.

### Step 1: Define the problem.

You are a data scientist with a financial services company. Specifically, you want to leverage data in order to identify potential customers.

If you are unfamiliar with "401(k)s" or "IRAs," these are two types of retirement accounts. Very broadly speaking:
- You can put money for retirement into both of these accounts.
- The money in these accounts gets invested and hopefully has a lot more money in it when you retire.
- These are a little different from regular bank accounts in that there are certain tax benefits to these accounts. Also, employers frequently match money that you put into a 401k.
- If you want to learn more about them, check out [this site](https://www.nerdwallet.com/article/ira-vs-401k-retirement-accounts).

We will tackle one regression problem and one classification problem today.
- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

Check out the data dictionary [here](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES).

#### NOTE: When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable. 

#### When predicting `e401k`, you may use the entire dataframe if you wish.

In [3]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import root_mean_squared_error

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, \
BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

### Step 2: Gather the data.

##### 1. Read in the data.

In [5]:
d401k_df = pd.read_csv('401ksubs.csv')

In [6]:
# review sample
d401k_df.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809


In [7]:
# check for null
d401k_df.isnull().sum()

e401k     0
inc       0
marr      0
male      0
age       0
fsize     0
nettfa    0
p401k     0
pira      0
incsq     0
agesq     0
dtype: int64

##### 2. What are 2-3 other variables that, if available, would be helpful to have?

- Debt: Total debt (loans, credit card debt)
- Savings Rate: Percentage of income saved each year
- Risk Tolerance: Individual's willingness to take on investment risk.

##### 3. Suppose a peer recommended putting `race` into your model in order to better predict who to target when advertising IRAs and 401(k)s. Why would this be an unethical decision?

It could lead to discriminatory practices, as certain racial groups might be excluded or targeted disproportionately, regardless of their actual financial needs or potential.

## Step 3: Explore the data.

##### 4. When attempting to predict income, which feature(s) would we reasonably not use? Why?

Calculate income by taking the square root of incsq, which is derived from inc squared.

##### 5. What two variables have already been created for us through feature engineering? Come up with a hypothesis as to why subject-matter experts may have done this.
> This need not be a "statistical hypothesis." Just brainstorm why SMEs (Subject Matter Experts) might have done this!

##### 6. Looking at the data dictionary, one variable description appears to be an error. What is this error, and what do you think the correct value would be?

## Step 4: Model the data. (Part 1: Regression Problem)

Recall:
- Problem: What features best predict one's income?
- When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable.

##### 7. List all modeling tactics we've learned that could be used to solve a regression problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific regression problem and explain why or why not.

Linear Regression: Suitable for linear relationships.
Logistic Regression: Ideal for classification multi catagories.
Decision Tree Regressor: Appropriate for multi catagories.
Gradient Boosting
Gradient Descent

##### 8. Regardless of your answer to number 7, fit at least one of each of the following models to attempt to solve the regression problem above:
    - a multiple linear regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend setting a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [19]:
# initialize values

X = d401k_df[['marr','male','age','agesq','fsize','nettfa']]
y = d401k_df['inc']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [41]:
# scale value
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [79]:
test_models = {
    'ADA': AdaBoostRegressor(),
    'BAG': BaggingRegressor(),
    'DCT': DecisionTreeRegressor(),
    'LR': LinearRegression(),
    'KNN': KNeighborsRegressor(),
    'RF': RandomForestRegressor(),
}

search_params = {
    'ADA': {'n_estimators': [10, 30, 30], 'learning_rate': [0.001, 0.01, 0.1]},
    'BAG': {'n_estimators': [10, 20, 30]},
    'DCT': {'max_depth': [None, 10, 20, 30]},
    'LR': {},
    'KNN': {'n_neighbors': [1, 3, 5]},
    'RF': {'n_estimators': [10, 20, 30], 'max_depth': [None, 5, 10]},
}

In [109]:
# testing model
for name, model in test_models.items():
    grid = GridSearchCV(estimator=model, param_grid=search_params[name])
    grid.fit(X_train_sc, y_train)
    print(f"Best Estimator of {model} is {grid.best_estimator_}")
    print(f"Best Score of {model} is {grid.best_score_}")
    print(f"Training Score of {model} is {grid.score(X_train_sc, y_train)}")
    print(f"Testing Score of {model} is {grid.score(X_test_sc, y_test)}")
    print('-'*80)

Best Estimator of AdaBoostRegressor() is AdaBoostRegressor(learning_rate=0.1, n_estimators=10)
Best Score of AdaBoostRegressor() is 0.363805228674792
Training Score of AdaBoostRegressor() is 0.3702223104232687
Testing Score of AdaBoostRegressor() is 0.3412004063138353
--------------------------------------------------------------------------------
Best Estimator of BaggingRegressor() is BaggingRegressor(n_estimators=30)
Best Score of BaggingRegressor() is 0.2794419641882649
Training Score of BaggingRegressor() is 0.887648435227115
Testing Score of BaggingRegressor() is 0.29712486845692454
--------------------------------------------------------------------------------
Best Estimator of DecisionTreeRegressor() is DecisionTreeRegressor(max_depth=10)
Best Score of DecisionTreeRegressor() is 0.17637352752617502
Training Score of DecisionTreeRegressor() is 0.5800285053196111
Testing Score of DecisionTreeRegressor() is 0.2534744399826425
------------------------------------------------------

##### 9. What is bootstrapping?

Resampling technique used to estimate the variability of a statistic use to estimate model performance ot find feature importance.

##### 10. What is the difference between a decision tree and a set of bagged decision trees? Be specific and precise!

- Decision trees are single models that can be lead to overfitting.
- Bagged decision trees are ensembles of multiple decision trees that can reduce overfitting and improve performance.

##### 11. What is the difference between a set of bagged decision trees and a random forest? Be specific and precise!

Bagging and random forest are both ensemble techniques that use decision trees.
- Bagging reduces variance by averaging multiple trees.
- Random forest further improves performance by introducing feature randomness.

##### 12. Why might a random forest be superior to a set of bagged decision trees?
> Hint: Consider the bias-variance tradeoff.

Random forests are generally superior to bagged decision trees because they introduce additional randomness by considering only a subset of features at each split, further reducing correlation between trees and improving 
performance.

## Step 5: Evaluate the model. (Part 1: Regression Problem)

##### 13. Using RMSE, evaluate each of the models you fit on both the training and testing data.

In [118]:
# based on best model from item 8
model = RandomForestRegressor(max_depth=5, n_estimators=30)
model.fit(X_train_sc, y_train)
y_train_pred = model.predict(X_train_sc)
y_test_pred = model.predict(X_test_sc)
print(f"Training RMSE is {root_mean_squared_error(y_train, y_train_pred)}")
print(f"Testing RMSE is {root_mean_squared_error(y_test, y_test_pred)}")

Training RMSE is 18.050230733394983
Testing RMSE is 19.127897723398632


##### 14. Based on training RMSE and testing RMSE, is there evidence of overfitting in any of your models? Which ones?

It doesn't overfit, as the RMSE remains relatively the sam.

##### 15. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 16. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 4: Model the data. (Part 2: Classification Problem)

Recall:
- Problem: Predict whether or not one is eligible for a 401k.
- When predicting `e401k`, you may use the entire dataframe if you wish.

##### 17. While you're allowed to use every variable in your dataframe, mention at least one disadvantage of using `p401k` in your model.

##### 18. List all modeling tactics we've learned that could be used to solve a classification problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific classification problem and explain why or why not.

##### 19. Regardless of your answer to number 18, fit at least one of each of the following models to attempt to solve the classification problem above:
    - a logistic regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend using a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

## Step 5: Evaluate the model. (Part 2: Classfication Problem)

##### 20. Suppose our "positive" class is that someone is eligible for a 401(k). What are our false positives? What are our false negatives?

##### 21. In this specific case, would we rather minimize false positives or minimize false negatives? Defend your choice.

##### 22. Suppose we wanted to optimize for the answer you provided in problem 21. Which metric would we optimize in this case?

##### 23. Suppose that instead of optimizing for the metric in problem 21, we wanted to balance our false positives and false negatives using `f1-score`. Why might [f1-score](https://en.wikipedia.org/wiki/F1_score) be an appropriate metric to use here?

##### 24. Using f1-score, evaluate each of the models you fit on both the training and testing data.

##### 25. Based on training f1-score and testing f1-score, is there evidence of overfitting in any of your models? Which ones?

##### 26. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 27. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 6: Answer the problem.

##### BONUS: Briefly summarize your answers to the regression and classification problems. Be sure to include any limitations or hesitations in your answer.

- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.